# Imports

In [1]:
#!pip install transformers datasets pandas

In [2]:

import pandas as pd
import datasets
from sklearn.model_selection import train_test_split
from datasets import Dataset, Features, ClassLabel, Value

/opt/homebrew/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get data

In [3]:
df = pd.read_pickle("DATA/reddit_wsb_sentiment.pkl")
df['created'] = pd.to_datetime(df['created'], unit="s")
titles = df["title"]
bodies = df["body"].dropna()
df["text"] = df["title"].str.cat(df["body"], sep=" ### ", na_rep="(Empty body)")

df.head()

,title,score,id,url,comms_num,created,body,timestamp,text,sentiment
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,2021-01-28 19:37:41,NaN,2021-01-28 21:37:41,"It's not about the money, it's about sending a...",0.0000
1,Math Professor Scott Steiner says the numbers ...,110,l6uibd,https://v.redd.it/ah50lyny62e61,23,2021-01-28 19:32:10,NaN,2021-01-28 21:32:10,Math Professor Scott Steiner says the numbers ...,-0.7034
2,Exit the system,0,l6uhhn,https://www.reddit.com/r/wallstreetbets/commen...,47,2021-01-28 19:30:35,The CEO of NASDAQ pushed to halt trading “to g...,2021-01-28 21:30:35,Exit the system ### The CEO of NASDAQ pushed t...,-0.4199
3,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,29,l6ugk6,https://sec.report/Document/0001193125-21-019848/,74,2021-01-28 19:28:57,NaN,2021-01-28 21:28:57,NEW SEC FILING FOR GME! CAN SOMEONE LESS RETAR...,-0.4822
4,"Not to distract from GME, just thought our AMC...",71,l6ufgy,https://i.redd.it/4h2sukb662e61.jpg,156,2021-01-28 19:26:56,NaN,2021-01-28 21:26:56,"Not to distract from GME, just thought our AMC...",0.2235


We'll use the following example sentence

In [4]:
sentence = df["text"].iloc[15]
sentence

'420 wasn’t a meme. GME 🚀 🚀 🚀 ### (Empty body)'

# Calculate simple target

In [5]:
df["labels"] = pd.qcut(df["sentiment"],[0,0.3,0.7,1], labels=["negative", "neutral", "positive"]).astype(str)
df["labels"].value_counts()

negative    26964
positive    14940
neutral      8166
Name: labels, dtype: int64

## Clean data
Not needed, these are Transformers!

In [6]:
df_train, df_test = train_test_split(df, random_state=22141)
df_train

,title,score,id,url,comms_num,created,body,timestamp,text,sentiment,labels
30426,This floppy dongus and the 69 price is the las...,19,ldvrax,https://i.redd.it/ew3dzgiwbuf61.jpg,6,2021-02-06 19:13:03,NaN,2021-02-06 21:13:03,This floppy dongus and the 69 price is the las...,0.0000,negative
16710,Brokerage discussion - finding a new broker th...,47,l7w5vv,https://www.reddit.com/r/wallstreetbets/commen...,21,2021-01-30 00:01:25,"Now that I am leaving Robinhood, I wanted to s...",2021-01-30 02:01:25,Brokerage discussion - finding a new broker th...,-0.7074,negative
35331,"💎💪🏽🦍 GME, BB and RKT",56,lqkr1i,https://i.redd.it/3vad1gg2v8j61.jpg,9,2021-02-23 23:17:49,NaN,2021-02-24 01:17:49,"💎💪🏽🦍 GME, BB and RKT ### (Empty body)",0.0000,negative
32167,Tattle tale falsely accuses guy of calling wsb...,37,ley0yt,https://greenwald.substack.com/p/the-journalis...,10,2021-02-08 07:01:35,NaN,2021-02-08 09:01:35,Tattle tale falsely accuses guy of calling wsb...,-0.3400,negative
32246,Saving Private WoooooG. I'm still down here in...,14694,lf1xx2,https://i.redd.it/v6jdhcc7z5g61.png,776,2021-02-08 10:29:45,NaN,2021-02-08 12:29:45,Saving Private WoooooG. I'm still down here in...,0.3506,neutral
...,...,...,...,...,...,...,...,...,...,...,...
34898,hodl,56,lroq45,https://i.redd.it/74pmihmm3ij61.png,16,2021-02-25 06:21:50,NaN,2021-02-25 08:21:50,hodl ### (Empty body),0.0000,negative
10891,Just bought my first share of gme. All I could...,1,l71iuq,https://i.redd.it/p4b05xovo3e61.jpg,1,2021-01-29 00:33:33,NaN,2021-01-29 02:33:33,Just bought my first share of gme. All I could...,0.8225,positive
47641,I lost so much money this year...Only thing sa...,17,nmsxye,https://i.redd.it/xe919w2ukt171.jpg,29,2021-05-28 16:12:48,NaN,2021-05-28 19:12:48,I lost so much money this year...Only thing sa...,-0.3182,negative
49765,Swerve Into VERV,0,o244cg,https://www.reddit.com/r/wallstreetbets/commen...,17,2021-06-18 02:29:25,**Tl;dr:** I like Verve Therapeutics. It coul...,2021-06-18 05:29:25,Swerve Into VERV ### **Tl;dr:** I like Verve T...,0.9928,positive


## Transformers model 1

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

preprocess_function({"text":[sentence]})

{'input_ids': [[101, 17442, 2347, 1521, 1056, 1037, 2033, 4168, 1012, 13938, 2063, 100, 100, 100, 1001, 1001, 1001, 1006, 4064, 2303, 1007, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [9]:
class_names = ["negative", "neutral", "positive"]
emotion_features = Features({'__index_level_0__': Value('string'), 'text': Value('string'), 'labels': ClassLabel(names=class_names)})


In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
train = Dataset.from_pandas(df_train[["text", "labels"]], features=emotion_features)
train

Dataset({
    features: ['__index_level_0__', 'text', 'labels'],
    num_rows: 37552
})

In [12]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_train

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:04<00:00,  9.03ba/s]


Dataset({
    features: ['__index_level_0__', 'text', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 37552
})

In [13]:

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [14]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    # eval_dataset=tokenized_imdb["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text, __index_level_0__. If text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 37552
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2347


Step,Training Loss


KeyboardInterrupt: 

This takes a long time. I trained this in the cloud instead, results are in the lecture powerpoint.

## Evaluate

In [ ]:
from evaluate import evaluator
from datasets import load_dataset
task_evaluator = evaluator("text-classification")

results = task_evaluator.compute(
    model_or_pipeline=model,
    data=tokenized_test,
    metric="accuracy",
    label_mapping={
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
    # label_mapping=emotion_features["labels"]._str2int,
    label_column="labels",
    n_resamples=1,
    random_state=0,
    tokenizer=tokenizer
)

In [ ]:
results